# Proposal 1

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import CEWCV1
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 20
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_1.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'ewc_lambda_l1': 0.1,
    'ewc_lambda_l2': 1.0,
    'threshold': 1e-05,
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loss_metrics(minibatch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(minibatch=True, experience=True, stream=True),
    gpu_usage_metrics(experience=True, stream=True, gpu_id=TORCH_DEVICE),
    disk_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True, stream=True),
    accuracy_matrix_metrics(),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(stream=True, num_classes=scenario.n_classes),
    
    loggers=[InteractiveLogger()]
)

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")
/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")


## Plugin defitinitions

In [10]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [11]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = CEWCV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [12]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/20 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:14<00:00, 31.72it/s]
Epoch 0 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 100.5000
	Loss_MB/train_phase/train_stream/Task000 = 10.0561
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 14.7829
100%|██████████| 469/469 [00:14<00:00, 32.66it/s]
Epoch 1 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 109.9000
	Loss_MB/train_phase/train_stream/Task000 = 15.2343
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 14.3652
100%|██████████| 469/469 [00:15<00:00, 30.07it/s]
Epoch 2 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 97.4000
	Loss_MB/train_phase/train_stream/Task000 = 19.4598
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 15.5967
100%|██████████| 469/469 [00:24<00:00, 19.52it/s]
Epoch 3 ended.
	CPUUsage_MB/train_phase/train

  5%|▌         | 1/20 [03:22<1:04:09, 202.60s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 43.9078
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x176964070>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9141, 0.1255, 0.1388, 0.1119, 0.1359, 0.1082, 0.1256, 0.1159, 0.1014,
         0.1219, 0.0808, 0.1192, 0.1151, 0.0877, 0.1053, 0.0931, 0.0891, 0.1330,
         0.1041, 0.1603],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

 10%|█         | 2/20 [06:46<1:00:57, 203.20s/it]

...

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 42.2972
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x178c35490>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9141, 0.1255, 0.1388, 0.1119, 0.1359, 0.1082, 0.1256, 0.1159, 0.1014,
         0.1219, 0.0808, 0.1192, 0.1151, 0.0877, 0.1053, 0.0931, 0.0891, 0.1330,
         0.1041, 0.1603],
        [0.8985, 0.9255, 0.1429, 0.1015, 0.1497, 0.0873, 0.0928, 0.1097, 0.1303,
         0.1751, 0.0922, 0.1149, 0.1422, 0.0924, 0.1413, 0.0856, 0.0897, 0.1130,
         0.1114, 0.1164],
        [0.8762, 0.8951, 0.9337, 0.0870, 0.1163, 0.1147, 0.0657, 0.1191, 0.1064,
         0.1478, 0.0815, 0.1166, 0.1331, 0.0843, 0.1804, 0.0722, 0.0609, 0.1452,
         0.1376, 0.0997],
        [0.8136, 0.8777, 0.9152, 0.9344, 0.0863, 0.0771, 0.0918, 0.1133, 0.1192,
         0.1243, 0.0787, 0.1207, 0.1724, 0.0736, 0.18

 30%|███       | 6/20 [23:11<59:13, 253.82s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 58.4797
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x1792222b0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9141, 0.1255, 0.1388, 0.1119, 0.1359, 0.1082, 0.1256, 0.1159, 0.1014,
         0.1219, 0.0808, 0.1192, 0.1151, 0.0877, 0.1053, 0.0931, 0.0891, 0.1330,
         0.1041, 0.1603],
        [0.8985, 0.9255, 0.1429, 0.1015, 0.1497, 0.0873, 0.0928, 0.1097, 0.1303,
         0.1751, 0.0922, 0.1149, 0.1422, 0.0924, 0.1413, 0.0856, 0.0897, 0.1130,
         0.1114, 0.1164],
        [0.8762, 0.8951, 0.9337, 0.0870, 0.1163, 0.1147, 0.0657, 0.1191, 0.1064,
         0.1478, 0.0815, 0.1166, 0.1331, 0.0843, 0.1804, 0.0722, 0.0609, 0.1452,
         0.1376, 0.0997],
        [0.8136, 0.8777, 0.9152, 0.9344, 0.0863, 0.0771, 0.0918, 0.1133, 0.1192,
         0.1243, 0.0787, 0.1207, 0.1724, 0.0736, 0.18

 35%|███▌      | 7/20 [27:09<53:51, 248.56s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 42.1695
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x17cd2ae80>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9141, 0.1255, 0.1388, 0.1119, 0.1359, 0.1082, 0.1256, 0.1159, 0.1014,
         0.1219, 0.0808, 0.1192, 0.1151, 0.0877, 0.1053, 0.0931, 0.0891, 0.1330,
         0.1041, 0.1603],
        [0.8985, 0.9255, 0.1429, 0.1015, 0.1497, 0.0873, 0.0928, 0.1097, 0.1303,
         0.1751, 0.0922, 0.1149, 0.1422, 0.0924, 0.1413, 0.0856, 0.0897, 0.1130,
         0.1114, 0.1164],
        [0.8762, 0.8951, 0.9337, 0.0870, 0.1163, 0.1147, 0.0657, 0.1191, 0.1064,
         0.1478, 0.0815, 0.1166, 0.1331, 0.0843, 0.1804, 0.0722, 0.0609, 0.1452,
         0.1376, 0.0997],
        [0.8136, 0.8777, 0.9152, 0.9344, 0.0863, 0.0771, 0.0918, 0.1133, 0.1192,
         0.1243, 0.0787, 0.1207, 0.1724, 0.0736, 0.18

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    ][1]
    losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Loss_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
all_loss_components = strategy.get_store_loss()

In [ ]:
weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    "class_accuracies": class_accuracies,
    "losses": losses,
    "train_cpu_usage": train_cpu_usage,
    "train_time_epoch": train_time_epoch,
    "train_running_time": train_running_time,
    "train_loss_epoch": train_loss_epoch,
    "all_loss_components": all_loss_components,
    "weights": weights
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))